# Problem

In a database, there lives a table. You want to compute a statistic using that
table. Ideally, you'd run your query like so:

```sql
SELECT
    SUM("field") AS statistic
FROM ginormous_table
WHERE "condition";
```

This query takes a long time because `ginormous_table` is ginormous. So you exploit a
feature supported by most databases: sampling. For Postgres DBs, the sampled query looks
something like:

```sql
SELECT
    SUM("field") * (100 / p) AS estimate  -- p% of table sampled, whatsupscale
FROM ginormous_table TABLESAMPLE BERNOULLI (p)
WHERE "condition";
```

The sampled query is significantly faster than the plain one. But anyone who has worked
with such queries knows that setting `p`—the percentage of the table to sample—is
tricky. Increasing `p` results in better estimates for the statistic, and may be
necessary when the `condition` is rarely met. But setting it too high causes the query
to take too long. What's the lowest `p` we can get away with?

# Formalization

I've seen this question pop up twice in work settings. The best answer is to do the hard
work of empirically evaluating different sampling percentages on real data. But let's
see if we can ballpark `p` given ballpark or worst-case guesses for certain statistics.

Some definitions:

* The table contains $N$ records (at the time that the query is run).

* We'll sample $n \leq N$ of them to compute the statistic. `p` = $n/N$.

* The random variable $X_i \sim \text{Bernoulli}(\lambda) \text{ i.i.d.}$ for
  $i = 1, 2, \dots, n$ indicates whether or not the $i$'th sampled record of the table
  matches the condition. It's safe to assume that they're independent b/c they're
  randomly sampled.

* The random variable $Y_i$ is the value of the field corresponding to sampled record
  $i$. We sum this field when computing the statistic of interest. If you're just trying
  to count, then $Y_i$ is not random; it's always $1$.
  * Assume they're i.i.d. w/ finite mean and variance Call $\mu_Y = \mathbb{E}(Y_i)$ and
    $\sigma^2_Y = \text{Var}(Y_i)$.
  * I'm pretty sure that it's very wrong to assume that $Y_i$'s are non-random in
    non-counting problems. There's variation in them through the random sample.

* The parameter of interest is what's expressed in the plain SQL query above:

$$
\begin{align*}
\mu &= \sum_{i \in \{i \: | \: X_i = 1 \} } Y_i \\
&= \sum_{i=1}^{N} X_i Y_i \\
&= (N \lambda) \mu_Y.
\end{align*}
$$

* Assume that $X_i \perp Y_j$ for all $i, j$. This assumption means that the number of
  records matching the condition is not correlated with the value of the field.
  * If they're positively correlated (assumption violated), but you don't really care
    about poorly estimating $\mu$ for a rare condition, then you shouldn't worry too
    much about setting $n$ too low. In other words, this analysis won't have much value
    to you.

* Let $M$ be a random variable for the number of records matching the condition *in the
  sample*:

  $$
  \begin{equation*}
  M = \sum_{i=1}^{n} X_i.
  \end{equation*}
  $$

  * In case it becomes useful, it's worth noting that in many actual queries, $n$ is
    pretty large and $\lambda$ is pretty small. This means $M$ is approximately
    Poisson-distributed, which is mathematically and computationally nicer than the
    Binomial distribution. I don't think we'll need this fact anywhere.

* The estimator for $\mu$ is:

$$
\begin{align*}
\hat{\mu} &= \Bigg( \sum_{i=1}^{n} X_i Y_i \Bigg) \frac{N}{n}\\
&= \Bigg( \sum_{i=1}^{M} Y_i \Bigg) \frac{N}{n}.
\end{align*}
$$

(I'm being sloppy with the subscripts. The $i$'s at this point are just for counting
purposes, which will be fine for the math b/c things are identically distributed.)

# Analysis

The estimator is the sum of a random number of random variables. So we'll need the facts
about expectation and variance [here](https://sites.stat.washington.edu/thompson/Stat395_09/Notes/week10.pdf).

$\hat{\mu}$ is unbiased:

$$
\begin{align*}
\mathbb{E}(\hat{\mu}) &= \frac{N}{n} \mathbb{E} \Bigg( \sum_{i=1}^{M} Y_i \Bigg) \\
&= \frac{N}{n} \Bigg( \sum_{i=1}^{M} \mathbb{E}(Y_i) \Bigg) \\
&= \frac{N}{n} (n \lambda) \mu_Y \\
&= (N \lambda) \mu_Y.
\end{align*}
$$

The variance is finite:

$$
\begin{align*}
\sigma^2 &= \text{Var}(\hat{\mu}) \\
&= \bigg( \frac{N}{n} \bigg)^2 \text{Var} \Bigg( \sum_{i=1}^{M}
Y_i \Bigg) \\
&= \bigg( \frac{N}{n} \bigg)^2 \bigg( \sigma^2_Y n \lambda + \mu^2_Y \big(
n\lambda(1-\lambda) \big) \bigg).
\end{align*}
$$

To calculate the lowest sample size needed to satisfy some error criterion, we typically
need to know the quantiles of the distribution of $\hat{\mu}$. I looked it up for the
case where the estimator is the sum of a random number of random variables, and [it's
not very
nice](https://math.stackexchange.com/questions/338129/random-sum-of-random-variables).
(If all $Y_i = 1$ b/c you're just counting records in a DB, then $\hat{\mu}$ is a
rescaled binomial.)

So let's just throw a bound at it. I'm thinking
[Chebyshev's](https://en.wikipedia.org/wiki/Chebyshev%27s_inequality). There are others,
but they're either worse (too high sample sizes) or too complicated for our purposes.

Let's modify Chebyshev's bound in two ways.

First, note that Chebyshev's bound is one on absolute error. Relative error seems more
interpretable for this application b/c we presumably will run the query over many
different conditions w/ many different $\mu$ values. If $\mu > 0$ (there are records
matching the condition in the table and their values are positive), then we can write a
bound on relative error:

$$
\begin{align*}
\Pr(|\hat{\mu} - \mu| \geq k\sigma) &\leq \frac{1}{k^2} \\
\Pr \bigg( \frac{|\hat{\mu} - \mu|}{\mu} \geq \frac{k\sigma}{\mu} \bigg) &\leq
\frac{1}{k^2}.
\end{align*}
$$

Next, it's easier for a user to input some $\epsilon > 0$ for the error instead of
$k$ standard deviations.

$$
\begin{align*}
\epsilon &= \frac{k \sigma}{\mu} \\
k &= \frac{\epsilon \mu}{\sigma} \\
\frac{1}{k^2} &= \bigg( \frac{\sigma}{\epsilon \mu} \bigg)^2.
\end{align*}
$$

So our $\epsilon$ + relative version of Chebyshev's inequality is:

$$
\begin{align*}
\Pr \bigg( \frac{|\hat{\mu} - \mu|}{\mu} \geq \epsilon \bigg) &\leq
\frac{\sigma^2}{\epsilon^2 \mu^2}.
\end{align*}
$$

Call the probability on LHS $p$.

To recover the sample size, plug in $\sigma^2 = \text{Var}(\hat{\mu})$. And we get our
sample size calculator:

$$
\begin{align*}
\epsilon^2 \mu^2 p &= \bigg( \frac{N}{n} \bigg)^2 \bigg( \sigma^2_Y n \lambda + \mu^2_Y
\big( n\lambda(1-\lambda) \big) \bigg) \\
\epsilon^2 \mu^2 p &= \frac{N^2}{n} \bigg( \sigma^2_Y \lambda + \mu^2_Y
\big( \lambda(1-\lambda) \big) \bigg) \\
n &= \frac{N^2 \bigg( \sigma^2_Y \lambda + \mu^2_Y
\big( \lambda(1-\lambda) \big) \bigg)}{\epsilon^2 \mu^2 p}
\end{align*}
$$

where:

* $\epsilon$ = maximum tolerated relative error ($ 0 < \epsilon < 1$)
* $p$ = maximum tolerated probability of the estimator resulting in $\epsilon$-or-worse
  relative error, i.e., if we ran the sampled query $100$ times, then expect the
  estimate to have $\epsilon$-or-worse relative error in at most $100p$ of them 
* $N$ = number of records in the table (at the time of the query)
* $\lambda$ = fraction of all records matching the condition
* $\sigma^2_Y$ = "true" variance of the field (which we're summing over in the query)
  for records matching the condition
* $\mu_Y$ = "true" mean of the field (which we're summing over in the query)
  for records matching the condition.

Drop the $\mu^2_Y$ term from the denominator if $\epsilon$ is the maximum tolerated
*absolute* error, not relative error.

If all $Y_i = 1$ b/c we're just counting (not summing a non-binary field), then $\mu_Y =
1$ and $\sigma^2_Y = 0$, so the sample size calculator collapses to:

$$
\begin{align*}
n &= \frac{N^2 \bigg( 0 \lambda + 1
\big( \lambda(1-\lambda) \big) \bigg)}{\epsilon^2 (N\lambda)^2 p} \\
&= \frac{1-\lambda}{\epsilon^2 \lambda p} \\
&= \bigg( \frac{1 - \lambda}{\lambda} \bigg) \bigg( \frac{1}{\epsilon^2} \bigg) \bigg(
\frac{1}{p} \bigg).
\end{align*}
$$

Sanity checking this formula is easy. Here are the checks:

- [ x ] The sample size $n$ is inversely related to our error tolerance, $\epsilon$ or
  $p$
- [ x ] The sample size $n$ is inversely related to the prevalence of matching records
  $\lambda$. 

# Implementation

In [ ]:
# TODO